## Vetle + Seb code

In [ ]:
%matplotlib inline

import os
import pandas as pd
import lasio as ls
#import selenium
#import phantomjs
import matplotlib.pyplot as plt
import numpy as np
import sys
import time
import dask.dataframe as dd
import dask.array as da
#import pyarrow.parquet as pq
import fastparquet
from functools import reduce

In [ ]:
import hvplot.pandas
import hvplot.dask
from hvplot import hvPlot
import holoviews as hv
from holoviews import opts, streams
from holoviews.plotting.links import DataLink
hv.extension('bokeh', logo=None)
from IPython.display import display, HTML
from bokeh.models.tools import LassoSelectTool, BoxSelectTool  # tools=['hover']
box_select = BoxSelectTool()
lasso_select = LassoSelectTool()
import panel as pn
pn.extension()
from IPython.display import display, HTML

client_secret = "Bw2w*31.CmbIiP.i2EILQ=HN@xr]yeu?"

display(HTML(data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 80%; }
    div#maintoolbar-container { width: 80%; }
</style>
"""))

In [ ]:
import random
import datashader as ds
from holoviews.streams import Selection1D

### Las generator

In [ ]:
filepath = '../Documentation/Well log LAS files/WKF W8A LWD VISION RM 0_5m Drilling.LAS'

In [ ]:
def las_to_df(filepath):
    las = ls.read(filepath)
    curve_df = las.df()
    curve_df = curve_df.replace(-999.25,np.nan)
    curve_df.reset_index(inplace=True)
    curve_df.rename(columns={curve_df.columns[0]:'Depth'}, inplace=True)
    las.curves[0].mnemonic = 'Depth'
    return curve_df, las

In [ ]:
df_realwell, las_realwell = las_to_df(filepath)

In [ ]:
# Create new df only containing relevant curves (9) within a start and stop depth:
def get_curves(df, curvenames, start_depth, stop_depth):
    df_new = df.loc[start_depth<=df['Depth']].loc[df['Depth']<stop_depth][curvenames]
    return df_new

In [ ]:
all_curves = ['Depth', 'GR_ARC', 'A16H', 'A28H', 'A40H', 'TNPH', 'ROP5_RM', 'RHOB', 'DCAV'] # Relevant curve names for the chosen well

start_depth = df_realwell['Depth'].iloc[0]
end_depth = df_realwell['Depth'].iloc[-1]
step_depth = df_realwell['Depth'].iloc[1] - df_realwell['Depth'].iloc[0]

df_curves = get_curves(df_realwell, all_curves, start_depth, end_depth) # New df containing specific curves and start and stop depths

#### Set input parameters

In [ ]:
Number_of_files = int(input("Number of files to generate: "))
depth_start = int(input("Depth reading start: "))
depth_end = int(input("Depth reading end: "))
depth_step = float(input("number of recordings per meter : ")) # 1=1 per m, 0.5=2 per m, 12= every 12 m
wellname = input('name of wells: ')
filename = input('name of files: ')

In [ ]:
len_data = len(np.arange(depth_start, depth_end, depth_step))

#### Create las-file from scratch

In [ ]:
def file_gen_input(wellname):
    file_arr = np.arange(1,Number_of_files+1,1)
    lasfile = ls.LASFile()
    from datetime import datetime
    lasfile.well.DATE = datetime.today().strftime('%Y-%m-%d %H:%M:%S')
    lasfile.well.WELL = wellname
    lasfile.params['CR8TOR'] = ls.HeaderItem('CR8TOR', value='Sebastian Aegidius')
    lasfile.other = 'LAS file creator'
    depth = np.arange(depth_start, depth_end, depth_step)
    lasfile.add_curve('DEPTH (m)', depth, unit = 'm')
    return lasfile, depth

In [ ]:
lasfile, depth = file_gen_input(wellname)

In [ ]:
def add_data(names, info_eq, fgi): # (Name of curve info, equation to produce data, file_gen_input_curve)
    data_eq = info_eq
    #fake[:3] = np.nan # Set first three values to NaN
    fgi.add_curve(names[0], data_eq, unit=names[1], descr=names[2]) # (parameter-name, datavalues, unit, description)
    return

#### Add multiple files, equations and data

In [ ]:
def multiple_files(Number_of_files, wellname = wellname):
    list_of_all_wells = []
    for i in range(Number_of_files):
        sample = file_gen_input(wellname+'_'+str(i))
        list_of_all_wells.append(sample[0])
    return list_of_all_wells

In [ ]:
params = [['GR', 'api', 'Gamma Ray'], 
          ['RD', 'ohm.m', 'Deep Resistivity'], 
          ['RM', 'ohm.m', 'Medium Resistivity'], 
          ['RS', 'ohm.m', 'Shallow Resistivity'], 
          ['NPHI', 'v/v', 'Neutron Porosity'], 
          ['RHOB', 'g/cm3', 'Density'], 
          ['DT', 'ms/ft', 'Sonic Velocity'], 
          ['CALI', 'inch', 'Caliper'], 
          ['BS', 'inch', 'Bit Size']
         ]

def eq_realdata(df):
    equation = [df[df.columns[0]].values, 
                df[df.columns[1]].values, 
                df[df.columns[2]].values,
                df[df.columns[3]].values, 
                df[df.columns[4]].values,
                df[df.columns[5]].values,
                df[df.columns[6]].values, 
                df[df.columns[7]].values, 
                df[df.columns[8]].values]
    equation = np.transpose(equation)
    equation = equation[0:len_data, :] # Only include equations from our chosen depth-interval
    return np.transpose(equation)

In [ ]:
def eq(lasfiles_object, eq_list):
    equations = []
    for i in range(len(lasfiles_object)): # Make unique equations for each las-file
        equations.append(eq_list)
    return equations

In [ ]:
def add_fakedata_multiple_files(fgi, equations, param_list):
    for las in range(len(fgi)): # Go through each las-file
        random = np.abs(np.random.gamma(shape=3, scale=2, size=len(depth))) # Random-data MUST be inside for-loop!
        #random_shift = 10 * np.random.sample() - 5
        random_shift = np.random.choice(np.concatenate([(1 + np.random.randn(5)*2), abs(np.random.randn(10)), (8 + np.random.randn(2) * 20)]))
        [add_data([param_list[i][0], param_list[i][1], param_list[i][2]], 
                      equations[las][i]*np.abs(np.random.gamma(shape=3, scale=2, size=len(depth)))
                      + equations[las][i]*random_shift, fgi[las]) for i in range(len(param_list))] # add 'random' data to a las-file
    return

In [ ]:
def write_to_las(fgi, filename):
    [fgi[i].write(filename+'_'+str(i)+'.las', version=2) for i in range(len(fgi))];
    return

In [ ]:
generated_wells = multiple_files(Number_of_files) # X number of las-files
add_fakedata_multiple_files(generated_wells, eq(generated_wells, eq_realdata(df_curves)), params) # Include some data in each file
write_to_las(generated_wells, filename) # Update las-files with the new data. TAKES ABOUT 20-30 SEC FOR 100 LAS-FILES.

#### Seb part:

In [ ]:
#File location and naming

def find_files(basedir, extfilter=''):
    files_ = []
    dirs = [basedir]
    while dirs:
        for e in os.scandir(dirs.pop()):
            if e.is_dir():
                dirs.append(e)
            else:
                if e.name.upper().endswith(extfilter.upper()):
                    files_.append(e.path)
    return files_

def path_leaf(path):
    for i in range(len(path)):
        path_name, file_name = os.path.split(path)
        file_name = file_name.replace('.las','')
    return file_name

def count_lines(filename):
    with open(filename) as f:
        return sum(1 for line in f)
    
if __name__ == '__main__':
    
    basedir = r"..\Documentation\Well log LAS files"
    start = time.perf_counter()
    files = find_files(basedir, '.las')
    stop = time.perf_counter()
    print('time to scan directory:', stop - start)
    print("Number of las files   :", len(files))
    print("Total lines           :", sum(map(count_lines, files)))
    #print(*files, sep='\n')
    #print()
    #file_name = [path_leaf(files[i]) for i in range(len(files))]
    #number_of_rows = [count_lines(files[i]) for i in range(len(files))]
    #print(*file_name, sep="\n")
    #print(*number_of_rows, sep='     ')
    #print()
    #print(number_of_rows[64])
    #print(file_name[64])

In [ ]:
def load_las_to_df(filepath_or_lasfile, generated=False):
    if (not generated):
        las = ls.read(filepath_or_lasfile)
        df = las.df()
    else:
        las = filepath_or_lasfile
        df = las.df() 
    df.reset_index(inplace=True)
    df.replace('-1.#IND', np.nan, inplace=True)
    df.replace('-1.#IO', np.nan, inplace=True)
    
    # Covnert objects to numerics:
    cols = df.columns[df.dtypes.eq(object)]
    df[cols] = df[cols].apply(pd.to_numeric, errors='coerce')
    
    df.replace(-999.25, np.nan, inplace=True)
    df.rename(columns={df.columns[0]:'DEPTH'}, inplace=True)
    df['WELLNAME'] = las.well.WELL.value
    return df

In [ ]:
##### only to show the list of dataframes
list_of_wells_df = [load_las_to_df(files[i]) for i in range(len(files))]

In [ ]:
df = pd.concat(list_of_wells_df, ignore_index=True)
df

In [ ]:
#writing dataframe to csv file
df.to_csv("Test_df.csv")

#Writing dataframe to parquet file
df.to_parquet('Test_df.parquet', compression = 'gzip', engine='fastparquet)

### Set some categories, and create dask from las-generated files

In [ ]:
generated_wells_df = [load_las_to_df(generated_wells[i], generated=True) for i in range(len(generated_wells))]

In [ ]:
big_df = pd.concat(generated_wells_df, ignore_index=True)

In [ ]:
nr_cat = 10
big_df['Category'] = np.concatenate([['cat'+str(i)]*int((len(big_df['WELLNAME'])/nr_cat)) for i in range(int(len(big_df['WELLNAME'])/int(len(big_df['WELLNAME'])/nr_cat)))]) # len(all_files)/20000 = 10 categories

In [ ]:
def get_numerics(df, depthcurvename): # only return the data that is numeric
    curve_list=list(df.columns[(df.dtypes.values == np.dtype('float64'))])
    curve_list.remove(depthcurvename) # Get rid of depth, so we have 9 remaining logs to plot
    return curve_list;

In [ ]:
num_curves = get_numerics(big_df, 'DEPTH')

In [ ]:
def lat_long(fgi, fgi_df):
    for i in range(len(fgi)):
        fgi[i].well['LATITUDE'] = ls.HeaderItem(mnemonic='LAT', unit='DEG', value=float('{:.8f}'.format((2 * np.random.sample() - 1)*90)), descr='Latitude')
        fgi[i].well['LONGITUDE'] = ls.HeaderItem(mnemonic='LONG', unit='DEG', value=float('{:.8f}'.format((2 * np.random.sample() - 1)*180)), descr='Longitude')
        start_idx = int(len(fgi_df)*i/len(fgi))
        stop_idx = int(len(fgi_df)*(i+1)/len(fgi))
        fgi_df.loc[start_idx:stop_idx, 'LATITUDE'] = fgi[i].well['LAT'].value # Sets specific latitude for each well
        fgi_df.loc[start_idx:stop_idx, 'LONGITUDE'] = fgi[i].well['LONG'].value # Sets specific longitude for each well
    return

In [ ]:
lat_long(generated_wells, big_df)

In [ ]:
big_df['Formation'] = ''
conditionFens = (big_df['DEPTH'] >= 1200) & (big_df['DEPTH'] < 1500)
conditionSogn = (big_df['DEPTH'] >= 1500) & (big_df['DEPTH'] < 1800)
big_df['Formation'] = big_df['Formation'].mask(conditionFens, 'Fensfjord')
big_df['Formation'] = big_df['Formation'].mask(conditionSogn, 'Sognefjord')

In [ ]:
formation_color = dict(pd.Series([{form: color for form, color in zip(big_df['Formation'].unique(), [0, 1, 2])}])[0])

In [ ]:
big_df['Formation_col'] = 0
for form, col in zip(formation_color.keys(), formation_color.values()):
    condition_form = big_df['Formation'] == form
    big_df['Formation_col'] = big_df['Formation_col'].mask(condition_form, col)

In [ ]:
dask_dummy = dd.from_pandas(big_df, npartitions=len(big_df['Category'].unique()))

In [ ]:
dask_dummy.to_parquet('big_df.parquet', compression = 'gzip', engine='fastparquet')
dask_df = dd.read_parquet('big_df.parquet', engine='fastparquet') 

In [ ]:
dask_df = dask_df.set_index('WELLNAME').persist()

## Scatterplot + Histogram

In [ ]:
from ipywidgets import interact
import ipywidgets as widgets
from holoviews.operation import histogram

@interact(paramX = widgets.Dropdown(
                    options = num_curves,
                    value = num_curves[0], # Set first well as default value
                    description = 'Param x:',
                    disabled = False),
          paramY = widgets.Dropdown(
                    options = num_curves,
                    value = num_curves[5], # Set second wel as default value
                    description = 'Param y:',
                    disabled = False),
          param_color = widgets.Dropdown(
                    options = num_curves + ['DEPTH'] + ['Form_color'] + ['Negative'], # all num_curves[i] contains same 9 parameters
                    value = None,
                    description = 'Color by:',
                    disabled = False),
          from_wellA = widgets.BoundedIntText(
                    value=1,
                    min=1,
                    max=len(dask_df.index.value_counts().compute()),
                    step=5,
                    description='A: From well nr:',
                    disabled=False
                ),
          to_wellA = widgets.BoundedIntText(
                    value=np.floor(len(dask_df.index.value_counts().compute())/2),
                    min=1,
                    max=len(dask_df.index.value_counts().compute()),
                    step=5,
                    description='A: To well nr:',
                    disabled=False
                ),
          from_wellB = widgets.BoundedIntText(
                    value=np.floor(len(dask_df.index.value_counts().compute())/2) + 1,
                    min=1,
                    max=len(dask_df.index.value_counts().compute()),
                    step=5,
                    description='B: From well nr:',
                    disabled=False
                ),
          to_wellB = widgets.BoundedIntText(
                    value=len(dask_df.index.value_counts().compute()),
                    min=1,
                    max=len(dask_df.index.value_counts().compute()),
                    step=5,
                    description='B: To well nr:',
                    disabled=False
                ),
          scatter_overlay = widgets.Checkbox(
                    value=False,
                    description='Overlay scatterplots',
                    disabled=False,
                    indent=True),
          datashade = widgets.Checkbox(
                    value=True,
                    description='Datashade',
                    disabled=False,
                    indent=True),
          random_data = widgets.ToggleButton(
                    value=False,
                    description='Random data',
                    disabled=False,
                    icon='check'),
)
def curve_plot_dask(paramX, paramY, param_color, from_wellA, to_wellA, from_wellB, to_wellB, scatter_overlay, datashade, random_data):
    if (random_data):
        paramX = np.random.choice(num_curves)
        paramY = np.random.choice(num_curves)
        param_color = np.random.choice(num_curves+['DEPTH']+['Blue']+['Negative'])
        from_wellA = np.random.choice(len(dask_df.index.value_counts().compute()))
        to_wellA = np.random.choice(np.arange(from_wellA, len(dask_df.index.value_counts().compute())))
        from_wellB = np.random.choice(len(dask_df.index.value_counts().compute()))
        to_wellB = np.random.choice(np.arange(from_wellB, len(dask_df.index.value_counts().compute())))
    
    from_wellA = from_wellA - 1 # Account for index
    to_wellA = to_wellA - 1
    from_wellB = from_wellB - 1
    to_wellB = to_wellB -1
    
    A_from_idx = big_df.loc[big_df['WELLNAME'] == wellname+'_'+str(from_wellA)].index[0]
    A_to_idx = big_df.loc[big_df['WELLNAME'] == wellname+'_'+str(to_wellA)].index[-1]
    B_from_idx = big_df.loc[big_df['WELLNAME'] == wellname+'_'+str(from_wellB)].index[0]
    B_to_idx = big_df.loc[big_df['WELLNAME'] == wellname+'_'+str(to_wellB)].index[-1]
    
    big_df['Negative'] = 0
    condition_neg = (big_df[paramX] < 0) | (big_df[paramY] < 0) 
    big_df['Negative'] = big_df['Negative'].mask(condition_neg, 1)
    
    logx = logy = False
    if (paramX == 'RS' or paramX == 'RM' or paramX == 'RD'): logx = True 
    if (paramY == 'RS' or paramY == 'RM' or paramY == 'RD'): logy = True
    cmapA = 'Winter'
    cmapB = 'Inferno'
    if (scatter_overlay):
        param_color = None
    aplot = big_df.iloc[A_from_idx:A_to_idx].hvplot.scatter(x=paramX, y=paramY, logx=logx, logy=logy,
                                                color=param_color, datashade=datashade, cmap=cmapA, label='Group A color: '+cmapA,
                                                height=400, width=600, title=f'A: Correlation of {paramX} and {paramY} for well {from_wellA+1} to well {to_wellA+1}')
    bplot = big_df.iloc[B_from_idx:B_to_idx].hvplot.scatter(x=paramX, y=paramY, logx=logx, logy=logy,
                                                color=param_color, datashade=datashade, cmap=cmapB, label='Group B color: '+cmapB,
                                                height=400, width=600, title=f'B: Correlation of {paramX} and {paramY} for well {from_wellB+1} to well {to_wellB+1}') 
    
    histograms = [ hv.NdOverlay({str(big_df.iloc[i].WELLNAME)+' to '+str(big_df.iloc[j].WELLNAME): 
                             big_df.iloc[i:j].hvplot.hist(y=param, logx=logx, logy=logy, invert=invert, bin_range=(min(dask_df[param]), max(dask_df[param])), 
                             bins=50) for i, j in zip([A_from_idx, B_from_idx], [A_to_idx, B_to_idx])}).opts(
                             'Histogram', height=400, width=600, alpha=0.6, muted_fill_alpha=0.15) for param, invert, logx, logy in zip([paramX, paramY], [False, True], [logx, False], [False, logy]) ]
    
    if (scatter_overlay): return pn.Column(pn.Row((aplot * bplot).opts(title=f'Correlation of {paramX} and {paramY} for A: {to_wellA - from_wellA} wells, and B: {to_wellB - from_wellB} wells')), pn.Row(histograms[0], histograms[1]))
    else: return pn.Column(pn.Row(aplot, bplot), pn.Row(histograms[0], histograms[1]))
    

In [ ]:
@interact(paramX = widgets.Dropdown(
                    options = num_curves,
                    value = num_curves[0], # Set first well as default value
                    description = 'Param x:',
                    disabled = False),
          paramY = widgets.Dropdown(
                    options = num_curves,
                    value = num_curves[1], # Set second wel as default value
                    description = 'Param y:',
                    disabled = False),
          param_color = widgets.Dropdown(
                    options = num_curves + ['DEPTH'] + ['Blue'], # all num_curves[i] contains same 9 parameters
                    value = 'Blue',
                    description = 'Color by:',
                    disabled = False),
          from_wellA = widgets.BoundedIntText(
                    value=0,
                    min=0,
                    max=len(dask_df.index.value_counts().compute()),
                    step=5,
                    description='A: From well nr:',
                    disabled=False
                ),
          to_wellA = widgets.BoundedIntText(
                    value=np.floor(len(dask_df.index.value_counts().compute())/2) - 1, # -1 to account for index
                    min=0,
                    max=len(dask_df.index.value_counts().compute()) - 1,
                    step=5,
                    description='A: To well nr:',
                    disabled=False
                ),
          to_wellB = widgets.BoundedIntText(
                    value=len(dask_df.index.value_counts().compute()) - 1,
                    min=0,
                    max=len(dask_df.index.value_counts().compute()) - 1,
                    step=5,
                    description='B: To well nr:',
                    disabled=False
                ),
          datashade = widgets.Checkbox(
                    value=True,
                    description='Datashade',
                    disabled=False,
                    indent=True),
)
def curve_plot_dask(paramX, paramY, param_color, from_wellA, to_wellA, to_wellB, datashade):
    aplot = dask_df.loc[wellname+'_'+str(from_wellA):wellname+'_'+str(to_wellA)].hvplot.scatter(x=paramX, y=paramY, 
                                                color=param_color, datashade=datashade, cmap='Hot',
                                                height=400, width=600)
    bplot = dask_df.loc[wellname+'_'+str(to_wellA + 1):wellname+'_'+str(to_wellB)].hvplot.scatter(x=paramX, y=paramY, 
                                                color=param_color, datashade=datashade, cmap='Winter',
                                                height=400, width=600)
    #histX = dask_df.hvplot.hist(y=paramX, bin_range=(min(dask_df[paramX]), max(dask_df[paramX])), bins=50)
    #histY = dask_df.hvplot.hist(y=paramY, bin_range=(min(dask_df[paramY]), max(dask_df[paramY])), bins=50)
    histX, histY = (histogram(bplot, bin_range=(min(dask_df[paramX]), max(dask_df[paramX])), dimension=dim) *
                histogram(aplot,  bin_range=(min(dask_df[paramY]), max(dask_df[paramY])), dimension=dim) 
                for dim in [paramX, paramY])
    #return aplot.hist(dimension=[paramX, paramY])
    #hv.output(aplot)
    composition = (bplot * aplot) << histY.opts(width=150, xlim=(min(dask_df[paramY])/10, max(dask_df[paramY])/10)) << histX.opts(height=150, xlim=(min(dask_df[paramX])/10, max(dask_df[paramX])/10))
    composition.opts(opts.Histogram(alpha=0.3))
    #return pn.Column(pn.Row(aplot, bplot), pn.Row(histX, histY))
    return composition

## Clustering

In [ ]:
import seaborn as sns
sns.set_color_codes()
import sklearn
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import scale # to scale our model

In [ ]:
list_files = [files[0], files[4], files[10], files[5]] # Choose three wells from our las-file folder
RHOB_names = ['RHOB', 'RHOZ', 'DEN', 'RHOB'] # Store the different density-names (NB: same order as list_files!)
NPHI_names = ['NPHI', 'NPHI', 'NPRL', 'NPHI'] # Store the different neutron-names (NB: same order as list_files!)
GR_names = ['GR', 'GR', 'GRGC', 'GR']

In [ ]:
def fill_NaN(df):
    for x in df.columns:
        if df[x].dtype == 'float64':
            len_NaN = len(df[x].isna())
            df[x].interpolate(method='linear', axis=0, limit=len_NaN, inplace=True) # Fill NaN values by doing linear interpolation with the preceding and upcoming value
            df[x].fillna(method='ffill', inplace=True)
            df[x].fillna(method='bfill', inplace=True)
            # Replace if any absurdly high or low values:
            if abs(np.min(df[x])) > 1000*np.mean(df[x]):
                df[x].replace(df[x].min(), np.mean(df[x]))
            if abs(np.max(df[x])) > 1000*np.mean(df[x]):
                df[x].replace(df[x].max(), np.mean(df[x]))
            df[x].fillna(np.mean(df[x]))
    return df

In [ ]:
# param1_names and param2_names must be a list of the two parameter names used by the files, in correct order!
def init_cluster(list_files, min_depth, max_depth, param1_names, param2_names, param1_realname, param2_realname):
    if (str(list_files[0]).lower().endswith('.las')): # if input is realwell
        df_and_las = [load_las_to_df(list_files[i]) for i in range(len(list_files))]
    else: # if input is generated well
        df_and_las = [load_las_to_df(list_files[i], generated=True) for i in range(len(list_files))]
    dfs = [df_and_las[i] for i in range(len(list_files))]
    
    for df in dfs:
        if ('DEPTH' not in df.columns):
            df.rename(columns={df.columns[0]:'DEPTH'}, inplace=True)
    [fill_NaN(i) for i in dfs]
    
    hold = []
    for df, param1_name, param2_name in zip(dfs, param1_names, param2_names):
        df = df.loc[df['DEPTH'] > min_depth]
        df = df.loc[df['DEPTH'] < max_depth]
        df.rename(columns={param1_name:param1_realname}, inplace=True)
        df.rename(columns={param2_name:param2_realname}, inplace=True)
        hold.append(df)
    dfs = hold
    return dfs, param1_realname, param2_realname

def KMeans_cluster(well_cluster):        
    multiple_wells = pd.concat(list(well_cluster[0]))
    
    new_df = pd.DataFrame({well_cluster[1]:multiple_wells[well_cluster[1]], well_cluster[2]:multiple_wells[well_cluster[2]]})
    new_array = np.array(new_df)
    
    X = scale(new_array)
    y = pd.concat([pd.Series(np.zeros(int(len(new_df)/3))), pd.Series(np.ones(int(len(new_df)/3))),
               pd.Series(np.ones(int(len(new_df)/3))*2)], ignore_index=True)
    
    clustering = KMeans(n_clusters=len(list_files))
    clustering.fit(X)
    clustering.labels_
    
    palette = sns.color_palette('deep', n_colors=np.unique(clustering.labels_).max() +1) # +1 because it returns index, but we need amount
    colors = [palette[x] for x in clustering.labels_]
    plt.scatter(x=new_df[well_cluster[1]], y=new_df[well_cluster[2]], color=colors)
    plt.xlabel(well_cluster[1])
    plt.ylabel(well_cluster[2])
    plt.title('Clustering of %d wells' %len(list_files))
    plt.show()
    return

In [ ]:
well_cluster = init_cluster(list_files, 1000, 2000, GR_names, NPHI_names, 'GR', 'NPHI')
plot_well_cluster = KMeans_cluster(well_cluster)

### Clustering with Dask

In [ ]:
import dask_ml.datasets
import dask_ml.cluster
import random

In [ ]:
from ipywidgets import interact
import ipywidgets as widgets

def interactive_scatter_cluster(big_df, n_clusters=4):
    start = time.perf_counter()
    @interact(param1 = widgets.Dropdown(
                        options = num_curves,
                        value = num_curves[0], # Set first well as default value
                        description = 'Param 1:',
                        disabled = False),
              param2 = widgets.Dropdown(
                        options = num_curves,
                        value = num_curves[1],
                        description = 'Param 2:',
                        disabled = False),
              from_well_nr = widgets.BoundedIntText(
                                value=0,
                                min=0,
                                max=len(big_df['WELLNAME'].unique()),
                                step=5,
                                description='From well nr:',
                                disabled=False
                            ),
              to_well_nr = widgets.BoundedIntText(
                                value=len(big_df['WELLNAME'].unique()),
                                min=0,
                                max=len(big_df['WELLNAME'].unique()),
                                step=5,
                                description='To well nr:',
                                disabled=False
                            )
    )
    def dask_KMeans(param1, param2, from_well_nr, to_well_nr):
        from_well_idx = from_well_nr * len(big_df.loc[big_df['WELLNAME'] == random.choice(big_df['WELLNAME'].unique())])
        to_well_idx = to_well_nr * len(big_df.loc[big_df['WELLNAME'] == random.choice(big_df['WELLNAME'].unique())])
        
        ml_df = pd.DataFrame({param1:big_df.iloc[from_well_idx:to_well_idx][param1], param2:big_df.iloc[from_well_idx:to_well_idx][param2]}) # create a (machine-learning) df containing two parameters
        ml_dd = dd.from_pandas(ml_df, npartitions=dask_df.npartitions)
        ml_array = ml_dd.to_dask_array(lengths=True) # lengths=True computes chunk size so the array is ready for machine-learning
        
        km = dask_ml.cluster.KMeans(n_clusters=n_clusters)
        km.fit(ml_array)
        
        ml_dd = ml_dd.assign(x = km.labels_).persist()
        scatt = ml_dd.hvplot.scatter(param1, param2, datashade=True, c=km.labels_, cmap='Viridis', 
                                     title='Clustering of %d wells' %(to_well_nr - from_well_nr))
        out = pn.Row(scatt)
        return hv.output(out)
    stop = time.perf_counter()
    print('Time:', stop - start)
    return

In [ ]:
interactive_scatter_cluster(big_df)

### Clustering with DBSCAN

In [ ]:
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from matplotlib import colors as mcolors
from sklearn.metrics import silhouette_samples, silhouette_score

def cluster_DBSCAN(big_df, param1, param2, eps=0.2, min_samples=10, from_well_nr=0, to_well_nr=10):
    from_well_idx = from_well_nr * len(big_df.loc[big_df['WELLNAME'] == random.choice(big_df['WELLNAME'].unique())])
    to_well_idx = to_well_nr * len(big_df.loc[big_df['WELLNAME'] == random.choice(big_df['WELLNAME'].unique())])
    
    ml_df = pd.DataFrame({param1:big_df.iloc[from_well_idx:to_well_idx][param1], param2:big_df.iloc[from_well_idx:to_well_idx][param2]}) # create a (machine-learning) df containing two parameters
    ml_dd = dd.from_pandas(ml_df, npartitions=dask_df.npartitions)
    ml_array = ml_dd.to_dask_array(lengths=True)
    
    X = StandardScaler().fit_transform(ml_array)
    print(X)
    # Compute DBSCAN
    db = DBSCAN(eps=eps, min_samples=min_samples).fit(X)
    core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
    core_samples_mask[db.core_sample_indices_] = True
    labels = db.labels_

    # Number of clusters in labels, ignoring noise if present.
    n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
    n_noise_ = list(labels).count(-1)
    
    print("Silhouette Coefficient: %0.3f"
      % silhouette_score(X, labels))
    
    # Plot results:
    unique_labels = set(labels)
    colors = [plt.cm.Accent(each)
              for each in np.linspace(0, 1, len(unique_labels))]
    #colors = cm.nipy_spectral(labels.astype(float) / len(unique_labels))
    colors = [mcolors.rgb2hex(c) for c in colors]
    all_scatt = []
    #hv_image = hv.Overlay([])
    for k, col in zip(unique_labels, colors):
        if k == -1:
            # Black used for outliers.
            col = mcolors.rgb2hex([0, 0, 0, 1])

        class_member_mask = (labels == k)
        part_of_core_or_not = [core_samples_mask, ~core_samples_mask]
        size = 10
        group_scatt = []
        
        for in_core_check in part_of_core_or_not:
            xy = ml_array[class_member_mask & in_core_check] # Clear part of cluster
            if (len(xy) > 0):
                ddf = dd.concat([dd.from_dask_array(i) for i in [xy[:,0], xy[:,1]]]) # !!!!!!!
                ddfA = da.from_array(ddf.reset_index()[0].compute()[0:int(len(ddf)/2)])
                ddfB = da.from_array(ddf.reset_index()[0].compute()[int(len(ddf)/2):])
                ddfA = list(ddfA.compute())
                ddfB = list(ddfB.compute())
                testy = pd.DataFrame({'a':ddfA, 'b':ddfB, 'c':[str(col)]*len(ddfA)})
                #plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
                #         markeredgecolor='k', markersize=14)
                group_scatt.append(testy.hvplot.scatter('a', 'b', c='c', s=size).opts(xlabel=param1, ylabel=param2, show_legend=True, colorbar=True))
            else:
                group_scatt.append(hv.Overlay([]))
            size*=0.25
        [all_scatt.append(group_scatt[i]) for i in range(len(group_scatt))]
    print(all_scatt)
    img = hv.Overlay(all_scatt).opts(toolbar='right')
    hv.output(img)
    #plt.title('Estimated number of clusters: %d' % n_clusters_)
    #plt.show()
    return

In [ ]:
cluster_DBSCAN(big_df, 'GR', 'RD', from_well_nr=60, to_well_nr=80)

## Dynamic Map

In [ ]:
from datashader.geo import lnglat_to_meters
from holoviews.element import tiles

In [ ]:
x, y = lnglat_to_meters(dask_df['LONGITUDE'], dask_df['LATITUDE'])
dask_df = dask_df.assign(x=x, y=y).persist()
big_df['x'] = x
big_df['y'] = y

In [ ]:
param_df = pd.DataFrame()
for param in range(len(num_curves)):
    param_curve = dask_df.iloc[:, param].compute()[::len(dask_df.loc['MapTest_0'])] # One of each param-value for each file
    param_df[param_curve.name] = param_curve.values
param_df['WELLNAME'] = dask_df.iloc[:,param].compute()[::len(dask_df.loc['MapTest_0'])].index.values
param_df['LATITUDE'] = [float(big_df.loc[big_df['WELLNAME'] == wellname]['LATITUDE'].unique()) for wellname in param_df['WELLNAME']]
param_df['LONGITUDE'] = [float(big_df.loc[big_df['WELLNAME'] == wellname]['LONGITUDE'].unique()) for wellname in param_df['WELLNAME']]

In [ ]:
param_df

In [ ]:
#big_df.loc[big_df['WELLNAME'] == 'MapTest_3'].loc[big_df['DEPTH'] >1800].loc[big_df['DEPTH'] <2100]['Formation']
param_df['Formation'] = [np.random.choice(big_df.loc[big_df['DEPTH'] == int(param_df.loc[param_df['WELLNAME'] == wellname]['DEPTH'].values)]['Formation']) for wellname in param_df['WELLNAME']]

In [ ]:
param_df['x'], param_df['y'] = lnglat_to_meters(param_df['LONGITUDE'], param_df['LATITUDE'])
param_df['RHOB_col'] = param_df['RHOB'].astype('int64') # ???

In [ ]:
param_df2 = param_df.copy()
param_df2['GR'] = param_df['GR'].where(param_df['GR'] > 5000)
high_GR = param_df2[param_df2['GR'] > 10000]
high_GR

In [ ]:
#param_points = param_df.hvplot.points(x='x', y='y', color='GR').opts(tools=['hover', 'tap'])
param_points = hv.Points(param_df, ['x', 'y'], 'GR').opts(color='GR', size=8, aspect='equal').opts(tools=['hover', 'tap'])

In [ ]:
def param_points2(param):
    return hv.Points(param_df, ['x', 'y'], param).opts(color=param, size=8, aspect='equal').opts(tools=['hover', 'tap'])

In [ ]:
hv.opts.defaults(hv.opts.Points(nonselection_alpha=0.4))
esri = tiles.ESRI().redim(x='x', y='y').opts(width=500, height=400, bgcolor='black')

In [ ]:
esri*param_points2('RHOB')

In [ ]:
selection_stream = streams.Selection1D(source=param_points)

In [ ]:
def labelled_callback(index):
    if len(index) == 0:
        return  hv.Text(x=min(param_df['x']),y=max(param_df['y']), text='Hei').opts(color='white')
    first_index = index[0] # Pick only the first one if multiple are selected
    row = param_df.iloc[first_index]
    text = '%s : lat: %d, long: %d' % (row.WELLNAME, row.LATITUDE, row.LONGITUDE)
    return hv.Text(x=row.x, y=row.y, text=text).opts(color='white')

labeller = hv.DynamicMap(labelled_callback, streams=[selection_stream])

In [ ]:
(esri * param_points * labeller).opts(hv.opts.Points(tools=['tap', 'hover'])).opts(width=700, height=600)

In [ ]:
param_df

In [ ]:
pointer = streams.PointerXY(x=0, y=0)

def cursor(x, y):
    return (hv.Box(x, y, 2e6/np.sqrt(2)) *
            hv.Ellipse(x, y, 2e6)).opts(
        hv.opts.Ellipse(color='yellow', alpha=0.7), hv.opts.Box(color='white', alpha=0.7)
    )
cursor_point = hv.DynamicMap(cursor, streams=[pointer])
cursor_point

In [ ]:
poly = hv.Ellipse(0, 0, 4)
poly_stream = streams.PolyDraw(source=poly, drag=True, num_objects=4,
                               show_vertices=True)
poly.opts(opts.Polygons(fill_alpha=0.3, active_tools=['poly_draw']))

In [ ]:
def mark_well(index):
    if len(index) == 0:
        return  hv.Overlay([]) # Empty overlay if we have not clicked on a well
    first_index = index[0] # Pick only the first one if multiple are selected
    row = param_df.iloc[first_index]
    return (hv.Box(row.x, row.y, 2e6/np.sqrt(2)) *
            hv.Ellipse(row.x, row.y, 2e6)).opts(
        hv.opts.Ellipse(color='yellow', alpha=0.7, line_width=3), hv.opts.Box(color='orange', alpha=0.7, line_width=3)
    )

well_marker = hv.DynamicMap(mark_well, streams=[selection_stream])

In [ ]:
(esri * param_points.opts(tools=['tap']) * well_marker).opts(width=700, height=600, max_width=700, max_height=600)

In [ ]:
def wells_around_point(df, lat, lon, degrees_dist=20):
    #if not index:
    #    return ds.iloc[[]]
    #row = param_points.data.iloc[index[0]]
    half_dist = degrees_dist/2
    #df = ds.data
    nearest = df[((df['LATITUDE'] - lat).abs() < half_dist) 
                 & ((df['LONGITUDE'] - lon).abs() < half_dist)]
    #return hv.Dataset(nearest)
    return nearest

In [ ]:
ds = hv.Dataset(param_df)
index_stream = Selection1D(source=param_points, index=[-10])

filtered_ds = ds.apply(wells_around_point, index=index_stream.param.index)

In [ ]:
def index_to_selection(indices, cache={}):
    if not indices: 
        return param_df.iloc[[]]
    index = indices[0]   # Pick only the first one if multiple are selected
    if index in cache: return cache[index]
    row = param_df.iloc[index]
    selected_df = wells_around_point(big_df, row.LATITUDE, row.LONGITUDE)
    cache[index] = selected_df # Save the selected wells in cache, so we don't have to reload the wells we select again
    return selected_df 

In [ ]:
hv.opts.defaults(
    hv.opts.Histogram(toolbar=None),
    hv.opts.Scatter(toolbar=None)
)

#def histogram(ds):
#    return ds.hvplot.hist(y='GR', bin_range=(0, 10), bins=20, color='red', width=400, height=250)

#def scatter(ds):
#    return ds.hvplot.scatter('NPHI', 'WELLNAME', color='green', width=400, height=250, padding=0.1)

def histogram_callback(index):
    title = 'Distribution of all wells within half a degree of selection'
    selected_df = index_to_selection(index)
    return selected_df.hvplot.hist(y='GR', bins=20, color='red', title=str(len(selected_df)))

histogram = hv.DynamicMap(histogram_callback, streams=[selection_stream])

def scatter_callback(index):
    title = 'Scattering of all wells within half a degree of selection '
    selected_df = index_to_selection(index)
    return selected_df.hvplot.scatter('GR', 'RHOB', color='WELLNAME', title=title)

scatter = hv.DynamicMap(scatter_callback, streams=[selection_stream])

def vline_callback(index):
    if not index:
        return hv.VLine(0)
    row = param_df.iloc[index[0]]
    return hv.VLine(row.NPHI).opts(line_width=1, color='black')

scatt_vline = hv.DynamicMap(vline_callback, streams=[index_stream])

#dynamic_scatter = param_df.apply(scatter, axis=1)
#dynamic_histogram = filtered_ds.apply(histogram)

In [ ]:
pn.Column(
    (esri * param_points.opts(tools=['tap']) * labeller * well_marker * cursor_point).opts(width=700, height=600, max_width=700, max_height=600),
    pn.Row(
        scatter * scatt_vline,
        histogram
    )
)